# Machine Learning, Exercise set #4

## 3. Boston housing data

Load the dataset from the text fileboston.txt.

In [1]:
import numpy as np

In [2]:
data = np.genfromtxt('data/boston.txt', skip_header=22)

Use the first 100 rows for testing, the next 50 rows for validation and the rest for training.

In [3]:
test, val, train = np.split(data, [100, 150])

### (a) Which column is the target? Which columns are the features?
<br>
We want to predict the price of the houses, i.e. Median value of owner-occupied homes in $ 1000's (MEDV) is our target value.

In [4]:
x_test, y_test = np.split(test, [-1], axis=1)
x_val, y_val = np.split(val, [-1], axis=1)
x_train, y_train = np.split(train, [-1], axis=1)

In [5]:
def MSE(y, y_pred):
    return np.mean((y - y_pred)**2)

Given the properties $x_1, x_2, ..., x_n \in \mathbb{R}^{13}$ and prices $y \in \mathbb{R}^n$, 
we want to find a good model (parameter $w \in \mathbb{R}^{13}$) that maps these properties onto price of the houses (MEDV), i.e.,

$$\begin{align*}
  \begin{bmatrix}
    \vert&\vert& &\vert \\
    x_1 & x_2 &\dots & x_{n} \\
    \vert& \vert& & \vert
  \end{bmatrix}^T w \approx y.
\end{align*}$$

### (b) using the MLE estimate
<br>
$$\begin{aligned}
w_\text{OLS} & = \text{arg}\min_w \sum_{i=1}^n(y_i-x_i^Tw)^2 \\
& = (X^TX)^{-1}X^Ty
\end{aligned}$$

which can also be written as a linear system:

$$Aw_\text{OLS} = b\\\begin{aligned}A & = X^TX\\b & = X^Ty\end{aligned}$$

In [6]:
def ordinary_least_squares(X, y):
    return np.linalg.solve(X.T @ X, X.T @ y)

In [7]:
w_ols = ordinary_least_squares(x_train, y_train)
y_pred = x_val @ w_ols
MSE(y_val, y_pred)

13.716605796368087

Report test error:

In [8]:
y_pred = x_test @ w_ols
MSE(y_test, y_pred)

10.626149240364173

### (c) using the ridge regression estimate
<br>
$$\begin{aligned}
w_\text{ridge} & = \text{arg}\min_w \sum_{i=1}^n(y_i-x_i^Tw)^2+\lambda \lVert w \rVert_2^2 \\
& = (\lambda I + X^TX)^{-1}X^Ty
\end{aligned}$$

which can also be written as a linear system:

$$Aw_\text{ridge} = b\\\begin{aligned}A & = \lambda I + X^TX\\b & = X^Ty\end{aligned}$$

In [9]:
def ridge_regression(X, y, lamda):
    A = X.T @ X + lamda * np.eye(X.shape[1])
    b = X.T @ y
    return np.linalg.solve(A, b)

In [10]:
best_val_err = np.inf
for lamda in np.logspace(-2., 2., num=50):
    w_ridge = ridge_regression(x_train, y_train, lamda)
    y_pred = x_val @ w_ridge
    val_err = MSE(y_val, y_pred)
        
    if val_err < best_val_err:
        best_val_err = val_err
        best_lamda = lamda
        best_w = w_ridge
        
print('Best hyperparameters: λ = {}\n'.format(best_lamda))

Best hyperparameters: λ = 7.196856730011514



Report test error:

In [11]:
y_pred = x_test @ best_w
MSE(y_test, y_pred)

10.49073419003747